In [22]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers


In [23]:
PINECONE_API="6ff60e9d-8723-4e7f-8ede-1f4841d6eb5c"
PINECONE_API_ENV="gcp-starter"


In [24]:
#extract data from pdf
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents =loader.load()
    return documents


In [25]:
extracted_data=load_pdf("D:\genai_medical_chatbot-main\genai_medical_chatbot-main\data")

In [26]:
#create chunks
def text_splitter(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

In [27]:
text_chunks=text_splitter(extracted_data)
print("length of my chunk",len(text_chunks))


length of my chunk 7020


In [28]:
#text_chunks

In [29]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [30]:
embeddings = download_hugging_face_embeddings()


In [31]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [32]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [33]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API
from langchain_pinecone import PineconeVectorStore
index_name = "medical"
#docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)



In [34]:
#If we already have an index we can load it like this
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)
query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE", metadata={'page': 130.0, 'source': '/workspaces/genai_medical_chatbot/data/Medical_book.pdf'}), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish

In [35]:
docs[0].page_content

"GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"

In [36]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [37]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [38]:
llm=CTransformers(model="D:\genai_medical_chatbot-main\genai_medical_chatbot-main\model\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [39]:
llm

CTransformers(client=<ctransformers.llm.LLM object at 0x0000026F8CA88550>, model='D:\\genai_medical_chatbot-main\\genai_medical_chatbot-main\\model\\llama-2-7b-chat.ggmlv3.q4_0.bin', model_type='llama', config={'max_new_tokens': 512, 'temperature': 0.8})

In [40]:
docsearch.as_retriever(search_kwargs={'k': 2})

VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000026F8CA722B0>, search_kwargs={'k': 2})

In [41]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [43]:

user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  Hypothermia is typically treated with warm fluids and/or blankets to help raise the body's core temperature. In some cases, medications such as glycerin or vasopressors may be used to help increase blood flow and improve circulation, which can help to warm the body. However, it's important to note that these medications should only be administered by a medical professional, as they can have serious side effects if not used properly.
